# **Classification of Web Documents using KNN**

## Importing Necessary Libraries

In [ ]:
import networkx as nx
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import re
from nltk.corpus import stopwords
!pip install pyspellchecker
from spellchecker import SpellChecker


import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Necessary Functions

### Remove Punctuations from text

In [ ]:
import string

def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    cleaned_text = text.translate(translator)
    return ' '.join(cleaned_text.split())

### Remove Numbers and Special Characters

In [ ]:
import re

def remove_numbers(text):
    pattern = r'[\d!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]+'
    cleaned_text = re.sub(pattern, '', text)
    return cleaned_text


### Correcting Misspelled Words

In [ ]:
!pip install pyspellchecker
from spellchecker import SpellChecker
def correct_spelling(text):
    spell = SpellChecker()
    corrected_words = [spell.correction(word) for word in text.split()]
    corrected_text = ' '.join(corrected_words)
    return corrected_text

### Remove Duplicate words within the text

In [ ]:
def remove_duplicates(text):
    word_list = text.split()
    unique_words = []
    for word in word_list:
        if word not in unique_words:
            unique_words.append(word)
    cleaned_text = ' '.join(unique_words)
    return cleaned_text

### Tokenizing the text

In [ ]:
def tokenization(text):
  return word_tokenize(text)

In [ ]:
from nltk.stem import WordNetLemmatizer

def lemmatize_text(text):
  lemmatizer = WordNetLemmatizer()
  words = tokenization(text)
  lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
  lemmatized_text = ' '.join(lemmatized_words)

  return lemmatized_text

### Removing Stopwords from the text

In [ ]:
import nltk
from nltk.corpus import stopwords

def remove_stopwords(text):
    words = text.split()
    filtered_words = []
    for word in words:
      word = word.lower()
      if word.lower() not in stopwords.words('english'):
          filtered_words.append(word)
    cleaned_text = ' '.join(filtered_words)

    return cleaned_text

### Stemming

In [ ]:
def stemTheWord(word):
  ps = PorterStemmer()
  if word.isalpha() and word.lower():
    return ps.stem(word)

In [ ]:
def stemming(tokens):
  ps = PorterStemmer()
  cleaned_tokens = [ps.stem(token) for token in tokens if token.isalpha() and token.lower() not in stopwords.words('english')]
  return cleaned_tokens

In [ ]:
def stem_the_graph(G_unique):
  stemmer = PorterStemmer()
  stemmed_nodes = {node: stemmer.stem(node) for node in G_unique.nodes()}
  G_stemmed = nx.relabel_nodes(G_unique, stemmed_nodes)
  return G_stemmed

## Getting Synonyms and Antonysm of the word

In [ ]:
import nltk
from nltk.corpus import wordnet

nltk.download('wordnet')

def getSynonymsAndAntonyms(word):
    synonyms = []
    antonyms = []
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonym = lemma.name().replace('_', ' ')
            if synonym not in synonyms:
                synonyms.append(synonym)
            for antonym in lemma.antonyms():
                antonym_name = antonym.name().replace('_', ' ')
                if antonym_name not in antonyms:
                    antonyms.append(antonym_name)
    return synonyms, antonyms

[nltk_data] Downloading package wordnet to /root/nltk_data...


### Tokens to Graphs

In [ ]:
def tokensToGraph(preprocessedTokens):
  DiGraph = nx.DiGraph()
  for i in range(len(preprocessedTokens)):
    if i != len(preprocessedTokens) - 1 :
        DiGraph.add_edge(preprocessedTokens[i], preprocessedTokens[i + 1])
    synonyms,antonyms = getSynonymsAndAntonyms(preprocessedTokens[i])
    for j in synonyms:
      if(len(j.split(" ")) == 1):
        DiGraph.add_edge(preprocessedTokens[i], j)
    for k in synonyms:
      if(len(k.split(" ")) == 1):
        DiGraph.add_edge(preprocessedTokens[i], k)
  return DiGraph

### Finding Similarity Between two graphs

In [ ]:
def commonEdgesThroughIntersection(G1, G2):
  common_edges = set(G1.edges).intersection(G2.edges)
  similarity = len(common_edges) / ((len(G1.edges) + len(G2.edges) - len(common_edges)))
  return similarity

In [ ]:
def jaccard_similarity(G1, G2):
    node_intersection = set(G1.nodes()) & set(G2.nodes())
    node_union = set(G1.nodes()) | set(G2.nodes())
    node_similarity = len(node_intersection) / len(node_union)

    edge_intersection = set(G1.edges()) & set(G2.edges())
    edge_union = set(G1.edges()) | set(G2.edges())
    edge_similarity = len(edge_intersection) / len(edge_union)

    return node_similarity + edge_similarity

### Returning maximum occuring label

In [ ]:
from collections import Counter

def max_occurrence_string(lst):
    # Count occurrences of each string in the list
    counts = Counter(lst)

    # Find the string with the maximum occurrence
    max_string = max(counts, key=counts.get)

    return max_string

In [ ]:
def returnLabel(data, similarGraphs):
  labels = []
  for title, similarity in similarGraphs.items():
    labels.append(data[title]["Label"])
  return max_occurrence_string(labels)


## Input Data

In [ ]:
import os

trainData = {}
# List all files in the directory
directory = "Train"
files = os.listdir(directory)
# Read contents of les:
for file in files:
  if os.path.isfile(directory + "/" + file):
        with open(directory + "/" + file, 'r') as f:
            text = f.read().split("\n")
            title = text[1]
            title = title[7:]
            tempDict = dict()
            tempDict["Label"] = text[0]
            tempDict["Text"] = text[2]
            tempDict["words"] = len(text[2].split(" "))
            trainData[title] = tempDict
            print(text[1])
            print("Topic: ", text[0])
            print("Total Words: ", tempDict["words"])
            print("--------------------------------------------")



Title: JPMorgan profit rises 6% even as interest income forecast falls short of predictions
Topic:  Business and Finance
Total Words:  640
--------------------------------------------
Title: How to Read the Bubbles In Your Coffee
Topic:  Food
Total Words:  279
--------------------------------------------
Title: 10 Benefits Of Using Active Socks In The Gym
Topic:  Health and Fitness
Total Words:  677
--------------------------------------------
Title: Apple's earnings power is once again being overlooked by investors, positioning the stock for 28% upside, BofA says
Topic:  Business and Finance
Total Words:  327
--------------------------------------------
Title: There are 4 reasons the stock market is poised to keep soaring to record highs, Goldman Sachs says
Topic:  Business and Finance
Total Words:  283
--------------------------------------------
Title: How AI is transforming business
Topic:  Business and Finance
Total Words:  812
--------------------------------------------
Title: W

In [ ]:
import os
testData = {}
# List all files in the directory
directory = "Test"
files = os.listdir(directory)
# Read contents of each file
for file in files:
    if os.path.isfile(directory + "/" + file):
        with open(directory + "/" + file, 'r') as f:
            text = f.read().split("\n")
            title = text[1]
            title = title[7:]
            tempDict = dict()
            tempDict["Label"] = text[0]
            tempDict["Text"] = text[2]
            tempDict["words"] = len(text[2].split(" "))
            testData[title] = tempDict
            print(text[1])
            print("Topic: ", text[0])
            print("Total Words: ", tempDict["words"])
            print("--------------------------------------------")

Title: 8 Ways To Improve Patient Care Quality
Topic:  Health and Fitness
Total Words:  1059
--------------------------------------------
Title: Biscuits And Gravy Egg Bake
Topic:  Food
Total Words:  259
--------------------------------------------
Title: WAYS TO EXERCISE AT HOME
Topic:  Health and Fitness
Total Words:  319
--------------------------------------------
Title: 6 Essential Tips For Setting Up Your Own Health And Fitness Blog
Topic:  Health and Fitness
Total Words:  654
--------------------------------------------
Title: Mental Health
Topic:  Health and Fitness
Total Words:  322
--------------------------------------------
Title: Chia seed benefits What you need to know
Topic:  Food
Total Words:  288
--------------------------------------------
Title: Quality In A Cup: 5 Influential Aspects Of Coffee Making
Topic:  Food
Total Words:  895
--------------------------------------------
Title: Health Benefits Of Cycling
Topic:  Health and Fitness
Total Words:  781
--------------

## Pre-Processing and Feature Extraction
### Pre-Processing


1.   Removing Stopwords
2.   Removing Numbers
3.   Remove Punctuations
4.   Removing Duplicates
5.   Lemmatizing Text
6.   Tokenization
7.   Creating Digraph with tokens
8.   Adding Synonyms and Antonyms
9.   Stemming the Graph


In [ ]:
def preprocessDocument(text):
  print("Removing Stopwords...")
  text = remove_stopwords(text)
  print("Removing Numbers...")
  text = remove_numbers(text)
  print("Removing Punctuations...")
  text = remove_punctuation(text)
  print("Removing Duplicates...")
  text = remove_duplicates(text)
  print("Lemmatizing...")
  text = lemmatize_text(text)
  print("Tokenizing...")
  tokens = tokenization(text)
  return tokens

In [ ]:
i = 0
for title, text in trainData.items():
  print("Pre-Processing Train Graph " + str(i + 1))
  preprocessedTokens = preprocessDocument(text["Text"])

  text["tokens"] = len(preprocessedTokens)
  print("Length of Tokens: ", text["tokens"])

  print("Creating Digraph with tokens adding synonyms and antonyms...")
  graph = tokensToGraph(preprocessedTokens)

  print("Stemming the Graph")
  text["Graph"] = stem_the_graph(graph)
  text["Nodes"] = len(text["Graph"].nodes())
  print("No. of Nodes: ", text["Nodes"])

  print("------------------------------------------")
  i = i + 1


Pre-Processing Train Graph 1
Removing Stopwords...
Removing Numbers...
Removing Punctuations...
Removing Duplicates...
Lemmatizing...
Tokenizing...
Length of Tokens:  358
Creating Digraph with tokens adding synonyms and antonyms...
Stemming the Graph
No. of Nodes:  1809
------------------------------------------
Pre-Processing Train Graph 2
Removing Stopwords...
Removing Numbers...
Removing Punctuations...
Removing Duplicates...
Lemmatizing...
Tokenizing...
Length of Tokens:  117
Creating Digraph with tokens adding synonyms and antonyms...
Stemming the Graph
No. of Nodes:  855
------------------------------------------
Pre-Processing Train Graph 3
Removing Stopwords...
Removing Numbers...
Removing Punctuations...
Removing Duplicates...
Lemmatizing...
Tokenizing...
Length of Tokens:  305
Creating Digraph with tokens adding synonyms and antonyms...
Stemming the Graph
No. of Nodes:  1914
------------------------------------------
Pre-Processing Train Graph 4
Removing Stopwords...
Removing

In [ ]:
j=0
for title, text in testData.items():
  print("Pre-Processing Test Graph " + str(j + 1))
  preprocessedTokens = preprocessDocument(text["Text"])

  text["tokens"] = len(preprocessedTokens)
  print("Length of Tokens: ", text["tokens"])

  print("Creating Digraph with tokens adding synonyms and antonyms...")
  graph = tokensToGraph(preprocessedTokens)

  print("Stemming the Graph")
  text["Graph"] = stem_the_graph(graph)
  text["Nodes"] = len(text["Graph"].nodes())
  print("No. of Nodes: ", text["Nodes"])

  print("------------------------------------------")
  j = j + 1


## Graph Visualization

In [ ]:
!pip install gravis
import gravis as gv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 659.1/659.1 kB 6.7 MB/s eta 0:00:00


In [28]:
for t,g in testData.items():
    print("Title: ", t)
    fig = gv.d3(g["Graph"],  use_node_size_normalization=True, node_size_normalization_max=30,
      use_edge_size_normalization=True, edge_size_data_source='weight', edge_curvature=0.3)
    fig.display(inline=True)
    break


Title:  8 Ways To Improve Patient Care Quality


## K-Nearest Neighbour Algorithm for classifying documents

In [29]:
def KNN(trainingGraphs,testGraphs, k):
  similarity = {}
  for title_test, testGraph in testGraphs.items():
    similarGraph = {}
    for title_train, trainGraph in trainingGraphs.items():
      similarityMeasure = jaccard_similarity(testGraph["Graph"], trainGraph["Graph"])
      similarGraph[title_train] = similarityMeasure
    similarGraph = {key: value for value, key in sorted([(value, key) for key, value in similarGraph.items()], reverse=True)}
    similarGraph =  dict(list(similarGraph.items())[:k])
    label = returnLabel(trainingGraphs, similarGraph)
    similarity[title_test] = label
  return similarity


## Training and Prediction

In [31]:
train = trainData
# print(train)
test = testData
model = KNN(train, test, 3)
print(model)

{'8 Ways To Improve Patient Care Quality': 'Health and Fitness', 'Biscuits And Gravy Egg Bake': 'Food', 'WAYS TO EXERCISE AT HOME': 'Health and Fitness', '6 Essential Tips For Setting Up Your Own Health And Fitness Blog': 'Health and Fitness', 'Mental Health': 'Health and Fitness', 'Chia seed benefits What you need to know': 'Food', 'Quality In A Cup: 5 Influential Aspects Of Coffee Making': 'Health and Fitness', 'Health Benefits Of Cycling': 'Health and Fitness', 'Tapping Into Flavor: Exploring The Secrets Of Making Maple Syrup': 'Food', 'Stock market today: Tech leads rebound as traders move on from CPI and brace for earnings': 'Business and Finance', 'Choosing The Perfect Cooking Class For You': 'Food', 'How Food May Improve Your Mood': 'Health and Fitness', "Jamie Dimon warns that markets are 'too happy' and failing to price in a possible US downturn": 'Business and Finance', 'Stock market today: futures mixed ahead of JPMorgan, BlackRock earnings': 'Business and Finance', 'Incredi

## Evaluation

In [32]:
correct = 0
wrong = 0

confusion_matrix = {}
total = len(testData)
for model_title, model_label in model.items():
  for test_title, text in testData.items():
    if model_title == test_title:
      if model_label == text["Label"]:
        correct = correct + 1
      else:
        print(model_title)
        wrong = wrong + 1

print("------------------------------------------------------------------------")
print("| Total Test Data: ", str(total))
print("------------------------------------------------------------------------")
print("| Correctly Classified: ", correct)
print("------------------------------------------------------------------------")
print("| Misclassified: ", wrong)
print("------------------------------------------------------------------------")


Quality In A Cup: 5 Influential Aspects Of Coffee Making
How Food May Improve Your Mood
------------------------------------------------------------------------
| Total Test Data:  17
------------------------------------------------------------------------
| Correctly Classified:  15
------------------------------------------------------------------------
| Misclassified:  2
------------------------------------------------------------------------


### `Confusion Matrix`

In [33]:
import numpy as np

def confusion_matrix(labels):

    all_labels = set([x[0] for x in labels])
    label_to_index = {label: idx for idx, label in enumerate(all_labels)}
    num_classes = len(all_labels)
    conf_matrix = np.zeros((num_classes, num_classes), dtype=int)
    for true, pred in labels:
         true_idx = label_to_index[true]
         pred_idx = label_to_index[pred]
         conf_matrix[true_idx, pred_idx] += 1
    return conf_matrix, label_to_index

# Example usage:
labels = [(values["Label"], _value ) for keys, values in testData.items() for _key, _value in model.items() if keys == _key ]
conf_matrix, label_to_index = confusion_matrix(labels)
print("Confusion Matrix:")
print(conf_matrix)
print("\nLabel to Index Mapping:")
print(label_to_index)

Confusion Matrix:
[[5 0 0]
 [0 5 2]
 [0 0 5]]

Label to Index Mapping:
{'Business and Finance': 0, 'Food': 1, 'Health and Fitness': 2}


### *Accuracy*



In [34]:
print("------------------------------------------------------------------------")
print("| Accuracy: "+ str(correct/total * 100.0) + " %")
print("------------------------------------------------------------------------")

------------------------------------------------------------------------
| Accuracy: 88.23529411764706 %
------------------------------------------------------------------------


### Recall

In [35]:
def calculate_recall(conf_matrix):
    recalls = []
    for i in range(len(conf_matrix)):
        tp = conf_matrix[i, i]
        fn = np.sum(conf_matrix[i]) - tp
        recall = tp / (tp + fn)
        recalls.append(recall)
    return recalls

all_labels = list(set([x[0] for x in labels]))
recall_scores = calculate_recall(conf_matrix)
print("------------------------------------------------------------------------")

for i in range(0,len(recall_scores)):
  print(f"| Recall for {all_labels[i]}: ", recall_scores[i])
  print("------------------------------------------------------------------------")



------------------------------------------------------------------------
| Recall for Business and Finance:  1.0
------------------------------------------------------------------------
| Recall for Food:  0.7142857142857143
------------------------------------------------------------------------
| Recall for Health and Fitness:  1.0
------------------------------------------------------------------------


### Precision

In [36]:
def calculate_precision(conf_matrix):
    precisions = []
    for i in range(len(conf_matrix)):
        tp = conf_matrix[i, i]
        fp = sum(conf_matrix[:, i]) - tp
        precision = tp / (tp + fp)
        precisions.append(precision)
    return precisions

all_labels = list(set([x[0] for x in labels]))
precision_scores = calculate_precision(conf_matrix)
print("------------------------------------------------------------------------")

for i in range(0,len(recall_scores)):
  print(f"| Precision for {all_labels[i]}: ", precision_scores[i])
  print("------------------------------------------------------------------------")


------------------------------------------------------------------------
| Precision for Business and Finance:  1.0
------------------------------------------------------------------------
| Precision for Food:  1.0
------------------------------------------------------------------------
| Precision for Health and Fitness:  0.7142857142857143
------------------------------------------------------------------------


### F1 Score

In [37]:
def calculate_f1_score(precision_scores, recall_scores):
    f1_scores = []
    for precision, recall in zip(precision_scores, recall_scores):
        f1_score = 2 * (precision * recall) / (precision + recall)
        f1_scores.append(f1_score)
    return f1_scores

all_labels = list(set([x[0] for x in labels]))
f1_scores = calculate_f1_score(precision_scores, recall_scores)
print("------------------------------------------------------------------------")

for i in range(0,len(recall_scores)):
  print(f"| F1 Score for {all_labels[i]}: ", f1_scores[i])
  print("------------------------------------------------------------------------")


------------------------------------------------------------------------
| F1 Score for Business and Finance:  1.0
------------------------------------------------------------------------
| F1 Score for Food:  0.8333333333333333
------------------------------------------------------------------------
| F1 Score for Health and Fitness:  0.8333333333333333
------------------------------------------------------------------------
